<a href="https://colab.research.google.com/github/satishchaudhary382/RFM/blob/master/stock_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [0]:
# Get the stock values 
#EBL, HBL, SHIV, NABIL
stocks = ['ebl','hbl','nabil','shiv']
for stock in stocks:
  exec(f"df_{stock} = stocks")

In [0]:
ebl = pd.read_excel("/content/stock.xlsx",sheet_name = 'ebl',names = ['Date','ebl_open','ebl_high','ebl_low','ebl_close','ebl_vloume'])
hbl = pd.read_excel("/content/stock.xlsx",sheet_name = 'hbl',names = ['Date','hbl_open','hbl_high','hbl_low','hbl_close','hbl_vloume'])
nabil = pd.read_excel("/content/stock.xlsx",sheet_name = 'nabil',names = ['Date','nabil_open','nabil_high','nabil_low','nabil_close','nabil_vloume'])
shiv = pd.read_excel("/content/stock.xlsx",sheet_name = 'shiv',names = ['Date','shiv_open','shiv_high','shiv_low','shiv_close','shiv_vloume'])

In [0]:
weights = np.array([0.2,0.2,0.2,0.2])

In [0]:
from functools import reduce
data_frames = [ebl,hbl,nabil,shiv]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), data_frames).fillna('void')

In [0]:
stock = df_merged[['Date','ebl_close','hbl_close','nabil_close','shiv_close']]
stock.set_index("Date", inplace = True)
stock.plot(figsize = (15,10))
plt.grid(True)

In [0]:
returns = stock.pct_change()
returns

In [0]:
returns = stock.pct_change()
returns

In [0]:
# calculate portfolio variance
port_variance = np.dot(weights.T,np.dot(cov_matrix_annual,weights))
port_variance

In [0]:
# calculate portfolio volitality aka standard deviation
port_volatility = np.sqrt(port_variance)
port_volatility

In [0]:
# calculate annual portfolio return 
portfolioSimpleAnnualReturn =np.sum(returns.mean()*weights)*245
portfolioSimpleAnnualReturn

In [0]:
# show expected annual return, volatility (risk)

percent_var = str(round(port_variance,2)*100)+"%"
percent_vol = str(round(port_volatility,2)*100)+"%"
percent_ret = str(round(portfolioSimpleAnnualReturn,2)*100)+"%"

print("Expected annual return: ", percent_ret)
print("Annual volatility/risk: ",percent_vol)
print("Annual variance: ", percent_var)

In [0]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [0]:
# Porfolio Optimization

# calculate expected returns and annualized sample covariance matrix of asset returns

mu = expected_returns.mean_historical_return(stock)
S = risk_models.sample_cov(stock)

# optimize for max sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights  =ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

In [0]:
# Get discrete allocation of each share per stock 
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(stock)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 30000)
allocation, leftover = da.lp_portfolio()

In [0]:
print("Discrete allocation:", allocation)
print("Funds remaining: Rs {:.2f}".format(leftover))